## Imports

In [8]:
from IPython.display import clear_output, display, Image
import os
import subprocess
import pandas as pd
import re
import cv2

In [9]:
#@title Clone the repository
!git clone https://github.com/xinyu1205/recognize-anything.git

%cd recognize-anything
!git checkout ec6b4241c5036e337a4543838deb9bff4990de97

Cloning into 'recognize-anything'...
remote: Enumerating objects: 694, done.
remote: Counting objects: 100% (404/404), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 694 (delta 284), reused 259 (delta 203), pack-reused 290
Receiving objects: 100% (694/694), 27.12 MiB | 29.70 MiB/s, done.
Resolving deltas: 100% (371/371), done.
/content/recognize-anything/recognize-anything
Note: switching to 'ec6b4241c5036e337a4543838deb9bff4990de97'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at ec6b4

In [10]:
#@title Install dependencies
!pip install timm transformers fairscale pycocoevalcap opencv-python

clear_output()

In [11]:
def download_checkpoints():
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
    if not os.path.exists(ram_weights_path):
        !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
    else:
        print("RAM weights already downloaded!")

In [12]:
download_checkpoints()
print('weights are downloaded!')

--2024-03-29 19:26:39--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 18.154.227.87, 18.154.227.7, 18.154.227.67, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.87|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2024-03-29 19:26:39--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/15c729c793af28b9d107c69f85836a1356d76ea830d4714699fb62e55fcc08ed?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ram_swin_large_14m.pth%3B+filename%3D%22ram_swin_large_14m.pth%22%3B&Ex

## Video to Frames

In [13]:
video_path = "/content/left_half.mp4"

output_dir = os.path.join("/content", "extracted_frames")
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
  print("Error opening video!")
  exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_count = 1
actual_count = 0

while True:
  ret, frame = cap.read()

  if not ret:
    print("Can't receive frame (stream end?). Exiting...")
    break

  # Save frame as numbered image
  if frame_count % fps == 0:
    image_path = os.path.join(output_dir, f"{frame_count}.jpg")
    cv2.imwrite(image_path, frame)
    actual_count += 1

  frame_count += 1

  if frame_count > total_frames:
    break

cap.release()

print(f"Extracted {actual_count} frames to {output_dir}")

Extracted 308 frames to /content/extracted_frames


## Inference using RAM

In [14]:
images_dir = "/content/extracted_frames"  # Update with your actual image directory

# Run inference for multiple images using RAM
output = subprocess.check_output([
    "python", "batch_inference.py",
    "--image-dir", images_dir,
    "--pretrained", "pretrained/ram_swin_large_14m.pth",
    "--model-type", "ram"
])

decoded_output = output.decode("utf-8")
image_tags = re.findall(r"'filepath': '.*/extracted_frames/(.*?\.jpg)', 'model_identified_tags': '(.*?)'", decoded_output)

df = pd.DataFrame(image_tags, columns=['Image', 'Tags'])

In [23]:
df

,Image,Tags
0,770.jpg,car | city street | cross | crosswalk | drive ...
1,4284.jpg,blue | car | city street | cross | crosswalk |...
2,2884.jpg,bag | city street | cross | crosswalk | paveme...
3,1624.jpg,city street | cross | crosswalk | dress | pers...
4,4256.jpg,car | city street | cross | crosswalk | person...
...,...,...
303,1638.jpg,city street | cross | crosswalk | person | int...
304,3920.jpg,car | city street | cross | crosswalk | person...
305,1456.jpg,building | car | city street | crack | crosswa...
306,1386.jpg,car | city street | cross | crosswalk | person...


In [25]:
decoded_output

'/encoder/layer/0/crossattention/self/query is tied\n/encoder/layer/0/crossattention/self/key is tied\n/encoder/layer/0/crossattention/self/value is tied\n/encoder/layer/0/crossattention/output/dense is tied\n/encoder/layer/0/crossattention/output/LayerNorm is tied\n/encoder/layer/0/intermediate/dense is tied\n/encoder/layer/0/output/dense is tied\n/encoder/layer/0/output/LayerNorm is tied\n/encoder/layer/1/crossattention/self/query is tied\n/encoder/layer/1/crossattention/self/key is tied\n/encoder/layer/1/crossattention/self/value is tied\n/encoder/layer/1/crossattention/output/dense is tied\n/encoder/layer/1/crossattention/output/LayerNorm is tied\n/encoder/layer/1/intermediate/dense is tied\n/encoder/layer/1/output/dense is tied\n/encoder/layer/1/output/LayerNorm is tied\n--------------\npretrained/ram_swin_large_14m.pth\n--------------\nload checkpoint from pretrained/ram_swin_large_14m.pth\nvit: swin_l\n{\'filepath\': \'/content/extracted_frames/770.jpg\', \'model_identified_tags